# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 13: Combinación y Agrupación de Datasets</center></h1>

Este notebook continua el trabajo anterior, con el objetivo que los estudiantes del curso aprendan diferentes comandos y técnicas en Python y, en particular, en Pandas, para la transformación de datos.

## 1. Concatenación de DataFrames

Veamos las diferentes formas de combinar dos DataFrames usando `concat`.


In [ ]:
import numpy as np
import pandas as pd

Creación de DataFrames

In [ ]:
df1 = pd.DataFrame(
    {
        "colA": ["A0", "A1", "A2", "A3"],
        "colB": ["B0", "B1", "B2", "B3"],
        "colC": ["C0", "C1", "C2", "C3"],
        "colD": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)


df2 = pd.DataFrame(
    {
        "colA": ["A4", "A5", "A6", "A7"],
        "colB": ["B4", "B5", "B6", "B7"],
        "colC": ["C4", "C5", "C6", "C7"],
        "colD": ["D4", "D5", "D6", "D7"],
    },
    index=[3,4,5,6],
)


df3 = pd.DataFrame(
    {
        "colA": ["A8", "A9", "A10", "A11"],
        "colB": ["B8", "B9", "B10", "B11"],
        "colC": ["C8", "C9", "C10", "C11"],
        "colD": ["D8", "D9", "D10", "D11"],
    },
    index=[0,1,2,3],
)


frames = [df1, df2, df3]

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df = pd.concat(frames, axis=1)
df

In [ ]:
df = pd.concat(frames, axis=0, join='outer').reset_index(drop=True)
df

### 2. Ejecicio: conectando datos de retail

Suponga que ud. tiene dos datasets correspondientes a los clientes de una tienda de retail, y todas las compras que ellos han realizado en distintos locales: `compras.csv`y `clientes.csv`.<br>

Además, cuenta con dos capas de datos vectoriales con la ubicación de los locales, y la zonificación de la ciudad.

Combine los datasets de manera adecuada para generar DataFrames que le permitan analizar las siguientes preguntas.

#### 2.1 ¿Cuánto es el monto total comprado por cada cliente registrado?

In [ ]:
clientes = pd.read_csv('data\\clientes.csv')
clientes

In [ ]:
clientes.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
compras = pd.read_csv('data\\compras.csv')
compras

In [ ]:
compras.drop(columns='Unnamed: 0',inplace=True)

¿Necesitamos conectar ambas bases de datos?

In [ ]:
clientes_compras = clientes.merge(compras, how='left', left_on='Rut', right_on='Rut')
clientes_compras

In [ ]:
clientes_compras.groupby(by='Rut').agg({'Monto':'sum'}).reset_index()

In [ ]:
pv = pd.pivot_table(clientes_compras, index='Rut', values='Monto', aggfunc="sum").reset_index()
pv

#### 2.2 ¿Cuál es la distribución de monto de ventas por edad?

In [ ]:
merged = compras.merge(clientes, how='inner')
merged

In [ ]:
clientes_compras.groupby(by=['Edad']).agg({'Monto':['sum','mean']})

#### 2.3 ¿Cuál es la distribución de monto de ventas por edad y local?

In [ ]:
clientes_compras.groupby(by=['Edad','Local']).agg({'Edad':'min','Monto':['sum','mean']})

#### 2.4 ¿Cuál es la fracción de ventas que representa cada cliente?

In [ ]:
def as_perc(value, total):
    return value/float(total)

In [ ]:
total_ventas = clientes_compras.Monto.sum()
clientes_compras['frac'] = clientes_compras[['Monto']].apply(as_perc,total=total_ventas)

In [ ]:
clientes_compras

In [ ]:
clientes_compras.groupby(by='Rut').agg({'frac':'sum'}).reset_index()

Algunos otros ejemplos

In [ ]:
pv = pd.pivot_table(clientes_compras,index='Rut',aggfunc="std", values='Monto')
pv

#### 2.5 ¿Hay alguna relación entre el tipo de zona donde se ubica un local, y las ventas del local?

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt 

locales = gpd.read_file('data\\locales.gpkg')
zonas = gpd.read_file('data\\zonas.gpkg')

In [ ]:
locales.crs

In [ ]:
zonas.crs

In [ ]:
zonas = zonas.to_crs(epsg=4326)
zonas.to_file('zonas.gpkg', driver='GPKG')
zonas.crs

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)

zonas.plot(ax=ax, figsize=(10,10), column='Tipo_zona', categorical=True, alpha=0.4, legend=True)
locales.plot(ax=ax, marker='o', color='k')

In [ ]:
locales_zonas = gpd.sjoin(locales, zonas, how='left', predicate='intersects', rsuffix='zona')
locales_zonas

In [ ]:
clientes_compras['ID_local'] = clientes_compras['Local'].str.replace('L','').astype('int')

In [ ]:
clientes_compras[clientes_compras['Local'].isna()]

In [ ]:
clientes_compras = clientes_compras.drop(9744)

In [ ]:
clientes_compras['ID_local'] = clientes_compras['Local'].str.replace('L','').astype('int')
clientes_compras.head()

In [ ]:
df = clientes_compras.merge(locales_zonas[['ID_local','Tipo_zona']], on='ID_local', how='left')
df

In [ ]:
pv = pd.pivot_table(df, index='Tipo_zona', values='Monto', aggfunc=["sum","mean","std","count"])
pv